**10) 한국어 전처리 패키지(Text Preprocessing Tools for Korean Text)**

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

1. PyKoSpacing

In [ ]:
sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
print(sent)

new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(new_sent)
print()

from pykospacing import spacing

kospacing_sent = spacing(new_sent)
print('원문과 PyKoSpacing 비교 ------------------------------------------')
print(sent)
print(kospacing_sent)
print()

2. Py-Hanspell

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
from hanspell import  spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
print(sent)
print()

spelled_sent = spell_checker.check(sent)
print(spelled_sent)
print()

hanspell_sent = spelled_sent.checked
print('맞춤법 및 띄어쓰기 보정 -------------------------------')
print(hanspell_sent)
print()
print()

print(new_sent)
print()
spelled_sent = spell_checker.check(new_sent)
print(spelled_sent)
print()

hanspell_sent = spelled_sent.checked
print('Py-Hanspell 과 PyKoSpacing 비교 ------------------------')
print(hanspell_sent)
print(kospacing_sent) # 앞서 사용한 kospacing 패키지에서 얻은 결과

3. SOYNLP를 이용한 단어 토큰화

[soynlp - Github](https://github.com/lovit/soynlp)

In [ ]:
!pip3 install soynlp
!pip3 install konlpy

1) 신조어 문제

In [ ]:
# import konlpy
from konlpy.tag import Okt
tokenizer = Okt()
print(tokenizer.morphs('에이비식스 이대휘 1월 최애돌 기부 요정'))
print()

2) 학습하기

In [ ]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

# 학습에 필요한 한국어 문서 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

# 훈련 데이터를 다수의 문서로 분리
corpus = DoublespaceLineCorpus("2016-10-20.txt") 
print(len(corpus))

# 상위 3개의 문서만 출력
i=0
for document in corpus:
  if len(document) > 0:
    print(document)
    print()
    i += 1
  if i == 3:
    break   

print()

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()


3) SOYNLP의 응집 확률(cohesion probability)

  - 응집 확률은 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
  - 이 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높습니다

In [ ]:
print("반포한         : " + str(word_score_table["반포한"].cohesion_forward))
print("반포한강       : " + str(word_score_table["반포한강"].cohesion_forward))
print("반포한강공     : " + str(word_score_table["반포한강공"].cohesion_forward))
print("반포한강공원   : " + str(word_score_table["반포한강공원"].cohesion_forward))
print("반포한강공원에 : " + str(word_score_table["반포한강공원에"].cohesion_forward))

4) SOYNLP의 브랜칭 엔트로피(branching entropy)
  -  주어진 문자열에서 얼마나 다음 문자가 등장할 수 있는지를 판단하는 척도
  - 브랜칭 엔트로피의 값은 하나의 완성된 단어에 가까워질수록 문맥으로 인해 점점 정확히 예측할 수 있게 되면서 점점 줄어드는 양상을 보입니다.

In [ ]:
print("디         : " + str(word_score_table["디"].right_branching_entropy))
print("디스       : " + str(word_score_table["디스"].right_branching_entropy))
print("디스플     : " + str(word_score_table["디스플"].right_branching_entropy))
print("디스플레   : " + str(word_score_table["디스플레"].right_branching_entropy))
print("디스플레이 : " + str(word_score_table["디스플레이"].right_branching_entropy))

5) SOYNLP의 L tokenizer

In [ ]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}

print(scores['국제사회'])
print(scores['우리'])
print(scores['노력'])
print(scores['범죄'])
print(scores['척결'])

l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

6) 최대 점수 토크나이저

In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer

print(scores['국'])
print(scores['국제'])
print(scores['국제사'])
print(scores['국제사회'])
print(scores['국제사회와'])
print(scores['와'])
print()
print(scores['우'])
print(scores['우리'])
print(scores['우리의'])
print(scores['의'])
print()

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

4. SOYNLP를 이용한 반복되는 문자 정제

In [ ]:
from soynlp.normalizer import *

print(emoticon_normalize('앜ㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print(emoticon_normalize('앜ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ이영화존잼쓰ㅠㅠㅠㅠㅠㅠㅠㅠ', num_repeats=2))
print()

print(repeat_normalize('와하하하하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하하하핫', num_repeats=2))
print(repeat_normalize('와하하하하핫', num_repeats=2))

5. Customized KoNLPy

In [ ]:
!pip3 install customized_konlpy

In [ ]:
# 형태소 분석 입력 : '은경이는 사무실로 갔습니다.'
# 형태소 분석 결과 : ['은', '경이', '는', '사무실', '로', '갔습니다', '.']

from ckonlpy.tag import Twitter

twitter = Twitter()
token = twitter.morphs('은경이는 사무실로 갔습니다.')
print('사전추가 전 --------------------------------')
print(token)
print()

twitter.add_dictionary('은경이', 'Noun')

token = twitter.morphs('은경이는 사무실로 갔습니다.')
print('사전추가 후 --------------------------------')
print(token)
print()